<left>FINM 32000 - Numerical Methods</left>
<left>Spring 2023</left>
<br>
<h1><center> Homework 1 </center></h1>
<center>Due - 23:59 [CST] April 7th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

### Imports

In [75]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from enum import Enum

# Problem 1

In [76]:
class OptionType(Enum):
    Put = "PUT"
    Call = "CALL"

In [77]:
class UpAndOutPut:
    
    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

In [78]:
hw1contract = UpAndOutPut(K=95, T=0.25, barrier=114, observationinterval=0.02)

In [79]:
class GBMdynamics: 
    
    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma
        
    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [80]:
hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [81]:
class Tree:
    
    def __init__(self, N):
        self.N = N
                
    def price_upandout(self, dynamics, contract): 
        
        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)
        
        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)  
        #Here I decided to make the SMALLER indexes in this array correspond to HIGHER S
        
        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")
            
        nu = dynamics.rGrow - dynamics.sigma**2 / 2
        Pu = ((dynamics.sigma**2 * deltat + nu**2 * deltat**2)/(deltax**2) + (nu * deltat)/deltax)/2
        Pd = ((dynamics.sigma**2 * deltat + nu**2 * deltat**2)/(deltax**2) - (nu * deltat)/deltax)/2
        Pm = 1 - ((dynamics.sigma**2 * deltat + nu**2 * deltat**2)/(deltax**2))

        optionprice = np.maximum(contract.K-Sgrid,0)   #an array of time-T option prices.
        
        #Next, induct backwards to time 0, updating the optionprice array 
        #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]

        tolerance = 1e-6 # for comparing floats
    
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)* deltat:
            # insert lines of code here if needed
            optionprice = np.exp(-dynamics.r * deltat) * (optionprice[:-2] * Pu +
                                                          optionprice[1:-1] * Pm +
                                                          optionprice[2:] * Pd)

            if abs(deltat % contract.observationinterval) < tolerance:
                if (len(optionprice) - 1)/2 < J:
                    pass
                else:
                    optionprice[:int((len(optionprice) - 1)/2 - J)] = 0.0


        return optionprice[0]   
        #The [0] is assuming that we are shrinking the optionprice array in each iteration of the loop,
        #until finally there is only 1 element in the array.
        #If instead you are keeping unchanged the size of the optionprice array in each iteration, 
        #then you need to change the [0] to a different index.


In [82]:
def Black_Scholes_Price (dynamics, contract, type):
    d_1 = (np.log(dynamics.S/contract.K) +
           ((dynamics.r) + dynamics.sigma ** 2 / 2) * contract.T) / (dynamics.sigma * np.sqrt(contract.T))

    d_2 = d_1 - (dynamics.sigma * np.sqrt(contract.T))

    if type == OptionType.Put:
        Price = norm.cdf(-d_2) * contract.K * np.exp(-dynamics.r * contract.T) - norm.cdf(-d_1) * dynamics.S
    elif type == OptionType.Call:
        Price = norm.cdf(d_1) * dynamics.S - norm.cdf(d_2) * contract.K * np.exp(-dynamics.r * contract.T)
    else:
        raise Exception("Select the appropriate type for the European Option (Put / Call)")

    return Price

## (a)

In [83]:
hw1tree=Tree(N=100000)

upandout_p = hw1tree.price_upandout(hw1dynamics, hw1contract)

upandout_p

5.5195384964568746

The price of the discretely monitored up-and-out barrier option, when $N$ is set to $100,000$, is approximately
$5.5195$.

## (b)

In [84]:
Black_Scholes_Price(hw1dynamics, hw1contract, OptionType.Put) - upandout_p

2.5672201005022544e-06

The payoff of long 1 up-and-in and long 1 up-and-out discretely monitored barrier option would be the same as the
payoff of long 1 plain vanilla put in the trinomial tree model.

Therefore, the price of the up-and-in option was calculated by subtracting the aforementioned up-and-out option price
from the plain vanilla put price, which was calculated through the Black-Scholes formula as the stock price strictly
follows a Geometric Brownian Motion.

The calculated price was very low as expected and was approximately $0.000003$

## (c)

### (c1)

### (c2)

# Problem 2

In [85]:
# uses the same GBMdynamics class as in Problem 1

In [86]:
class CallOption:
    
    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

    def BSprice(self, dynamics):
        # ignores self.price if given, because this function calculates price based on the dynamics 
        
        F = dynamics.S*np.exp(dynamics.rGrow*self.T)
        sd = dynamics.sigma*np.sqrt(self.T)
        d1 = np.log(F/self.K)/sd+sd/2
        d2 = d1-sd
        return np.exp(-dynamics.r*self.T)*(F*norm.cdf(d1)-self.K*norm.cdf(d2))
    
    def IV(self, dynamics):
        # ignores dynamics.sigma, because this function solves for sigma.  
        
        if self.price is None: 
            raise ValueError('Contract price must be given')
    
        df = np.exp(-dynamics.r*self.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-self.K)*df])
        C = self.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan 

        dytry = dynamics
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        dytry.sigma = 0.2
        while self.BSprice(dytry)>C:
            dytry.sigma /= 2
        while self.BSprice(dytry)<C:
            dytry.sigma *= 2
        hi = dytry.sigma
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above. 
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval, 
        # for purposes of this homework, you may either (A) write your own bisection algorithm, 
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution 
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).  
        # However, if you are able to bound the solution (of a single-variable problem), 
        # then bisection or Brent will be more reliable.

        impliedVolatility =      # you fill this in, using bisect or brentq imported from scipy.optimize,
                                 # or by writing your own bisection algorithm.
        return impliedVolatility


SyntaxError: invalid syntax (3742113503.py, line 56)

In [ ]:
#Test the BSprice function
dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract2 = CallOption(K=100, T=0.5)
contract2.BSprice(dynamics2)

In [ ]:
#Test the IV function
contract2.price = 12
contract2.IV(dynamics1)